In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os

os.environ['KAGGLE_USERNAME'] = 'sameerkumarchauhan'
os.environ['KAGGLE_KEY'] = 'c7b8f5db493592c1eb1892453cccfb47'


In [3]:
!kaggle competitions download -c store-sales-time-series-forecasting

!unzip /kaggle/working/store-sales-time-series-forecasting.zip

 94%|███████████████████████████████████▌  | 20.0M/21.4M [00:01<00:00, 24.8MB/s]
100%|██████████████████████████████████████| 21.4M/21.4M [00:01<00:00, 14.9MB/s]
Archive:  /kaggle/working/store-sales-time-series-forecasting.zip
  inflating: holidays_events.csv     
  inflating: oil.csv                 
  inflating: sample_submission.csv   
  inflating: stores.csv              
  inflating: test.csv                
  inflating: train.csv               
  inflating: transactions.csv        


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [5]:
# Load the datasets
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
stores_data = pd.read_csv('stores.csv')
oil_data = pd.read_csv('oil.csv')
holidays_events_data = pd.read_csv('holidays_events.csv')


In [6]:
display(train_data.head())
display(test_data.head())

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [7]:
# Convert 'date' column to datetime format
train_data['date'] = pd.to_datetime(train_data['date'])

# Extract date features
train_data['year'] = train_data['date'].dt.year
train_data['month'] = train_data['date'].dt.month
train_data['day'] = train_data['date'].dt.day
train_data['day_of_week'] = train_data['date'].dt.dayofweek


# Convert 'date' column to datetime format
test_data['date'] = pd.to_datetime(test_data['date'])

# Extract date features
test_data['year'] = test_data['date'].dt.year
test_data['month'] = test_data['date'].dt.month
test_data['day'] = test_data['date'].dt.day
test_data['day_of_week'] = test_data['date'].dt.dayofweek

In [8]:
display(train_data.head())
display(test_data.head())

,id,date,store_nbr,family,sales,onpromotion,year,month,day,day_of_week
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,2013,1,1,1
1,1,2013-01-01,1,BABY CARE,0.0,0,2013,1,1,1
2,2,2013-01-01,1,BEAUTY,0.0,0,2013,1,1,1
3,3,2013-01-01,1,BEVERAGES,0.0,0,2013,1,1,1
4,4,2013-01-01,1,BOOKS,0.0,0,2013,1,1,1


,id,date,store_nbr,family,onpromotion,year,month,day,day_of_week
0,3000888,2017-08-16,1,AUTOMOTIVE,0,2017,8,16,2
1,3000889,2017-08-16,1,BABY CARE,0,2017,8,16,2
2,3000890,2017-08-16,1,BEAUTY,2,2017,8,16,2
3,3000891,2017-08-16,1,BEVERAGES,20,2017,8,16,2
4,3000892,2017-08-16,1,BOOKS,0,2017,8,16,2


In [9]:
train_data = pd.get_dummies(train_data, columns=['family'], drop_first=True)
test_data = pd.get_dummies(test_data, columns=['family'], drop_first=True)


In [10]:
X = train_data.drop(columns = ['id', 'date', 'sales'])
y = train_data['sales']

test = test_data.drop(columns = ['id', 'date'])

In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(X)

X_scaled = scaler.transform(X)

test_scaled = scaler.transform(test)

In [32]:
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split

# model = RandomForestRegressor(n_estimators=20, random_state=42)
model = RandomForestRegressor(n_estimators=80, random_state=42)

X_train, X_valid, y_train, y_valid = train_test_split(X_scaled, y, test_size=0.2, random_state=0)

In [33]:
model.fit(X_train, y_train)

RandomForestRegressor(n_estimators=80, random_state=42)

In [34]:
y_pred = model.predict(X_valid)

In [35]:

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

mae = mean_absolute_error(y_valid, y_pred)
rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
print(f'Validation RMSE with selected features: {rmse}')
print(f'Mean Absolute Error: {mae:.2f}')

Validation RMSE with selected features: 235.6454740453702
Mean Absolute Error: 47.26


In [36]:
pred = model.predict(test_scaled)

id_start = 3000888
id_column = np.arange(id_start, id_start + len(pred))


submission = pd.DataFrame({
    'id': id_column,
    'sales': pred
})

In [37]:
submission.to_csv("submission.csv", index=False)


In [38]:
submission

,id,sales
0,3000888,4.737500
1,3000889,0.000000
2,3000890,6.475000
3,3000891,2311.100000
4,3000892,0.312500
...,...,...
28507,3029395,311.141113
28508,3029396,108.454588
28509,3029397,1247.488650
28510,3029398,133.850000
